<a href="https://colab.research.google.com/github/kbradbery/AI-Writer/blob/main/Shared_SERP_Based_Clustering_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import pandas as pd
import requests
import json

# Review the imported Excel file.

In [ ]:
file_path = '/content/inspection software.xlsx'

terms_df = pd.read_excel(file_path)
terms_df

,#,Keyword,Country,Difficulty,Volume,CPC,CPS,Parent Keyword,Last Update,SERP Features,Global volume,Traffic potential
0,1,inspection software,us,37.0,400.0,7.0,1.2,spectora,2024-02-15 08:47:28,"Sitelinks,People also ask,Bottom ads,Paid site...",1700.0,6900.0
1,2,safety inspection software,us,9.0,150.0,18.0,NaN,safety inspection software,2024-02-19 09:53:05,"Top ads,Paid sitelinks,Image pack,Bottom ads",700.0,90.0
2,3,inspection management software,us,7.0,150.0,10.0,NaN,inspection management software,2024-02-09 08:25:53,"People also ask,Image pack",800.0,150.0
3,4,quality inspection software,us,3.0,150.0,14.0,NaN,quality inspection software,2024-02-12 21:37:26,"People also ask,Videos",700.0,30.0
4,5,inspection report software,us,19.0,90.0,10.0,NaN,spectora,2024-02-14 10:03:59,"Top ads,Sitelinks,People also ask,Videos,Image...",350.0,6900.0
...,...,...,...,...,...,...,...,...,...,...,...,...
538,539,inspection and walkthrough in software testing,us,NaN,NaN,0.0,NaN,NaN,NaT,NaN,40.0,NaN
539,540,sprinkler inspection software download,us,NaN,NaN,0.0,NaN,NaN,NaT,NaN,30.0,NaN
540,541,renewable energy inspection software,us,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
541,542,railway train inspection software,us,NaN,NaN,0.0,NaN,NaN,NaT,NaN,10.0,NaN


# Clean Excelfile, and make it ready for Scraping

In [ ]:
def clean_excel_data(file_path):
    # Read the Excel file into a DataFrame
    terms_df = pd.read_excel(file_path)

    # Select only the 'Keyword' and 'Volume' columns and rename 'Keyword' to 'query'
    new_df = (
        terms_df[['Keyword', 'Volume']]
        .rename(columns={'Keyword': 'query'})
    )

    # Fill NaN values in 'Volume' column with 0 and convert 'Volume' to integer
    new_df['Volume'] = new_df['Volume'].fillna(0).astype(int)

    # Remove non-alphanumeric characters from 'query' column and filter out short queries
    new_df['query'] = new_df['query'].astype(str).str.replace('[^a-zA-Z0-9 ]', '', regex=True)
    new_df['query_length'] = new_df['query'].apply(len)
    new_df = new_df[new_df['query_length'] > 3]

    # Remove the 'query_length' column
    new_df = new_df.drop(columns=['query_length'])

    return new_df

# Example usage:
file_path = '/content/inspection software.xlsx'
keywords_df = clean_excel_data(file_path)
keywords_df

,query,Volume
0,inspection software,400
1,safety inspection software,150
2,inspection management software,150
3,quality inspection software,150
4,inspection report software,90
...,...,...
538,inspection and walkthrough in software testing,0
539,sprinkler inspection software download,0
540,renewable energy inspection software,0
541,railway train inspection software,0


# ValueSERP API Scraping

In [ ]:
import requests
import pandas as pd
import time

def create_batch(batch_name, api_key):
    body = {
        "name": batch_name,
        "enabled": True,
        "schedule_type" : "manual",
        "priority": "highest",
        "notification_email": "mihir23192@gmail.com",
    }

    api_result = requests.post(f'https://api.valueserp.com/batches?api_key={api_key}', json=body)
    api_response = api_result.json()

    return api_response['batch']['id']

def add_search_queries(batch_id, df, api_key, search_location, gl, hl, google_domain):
    batch_size = 1000  # Set the batch size to 1000 queries per request
    #queries = df['query'].tolist()
    # Ensure all queries are strings and not empty
    queries = [q for q in df['query'].tolist() if q and isinstance(q, str)]
    batches = [queries[i:i+batch_size] for i in range(0, len(queries), batch_size)]

    for i, batch in enumerate(batches):
        body = {"searches": []}

        for query in batch:
            search_entry = {
                "q": query,
                "location": search_location,
                'gl': gl,
                'hl': hl,
                "google_domain": google_domain
            }
            body["searches"].append(search_entry)

        api_result = requests.put(f'https://api.valueserp.com/batches/{batch_id}?api_key={api_key}', json=body)
        print(body)
        print(api_result)
        api_response = api_result.json()
        print(api_response)
    return api_response  # You can return the API response here if needed

def start_batch(batch_id, api_key):
    params = {'api_key': str(api_key)}
    api_result = requests.get(f'https://api.valueserp.com/batches/{batch_id}/start', params=params)
    api_response = api_result.json()
    return api_response

def get_result_set(batch_id, api_key):
    params = {'api_key': api_key}
    api_result = requests.get(f'https://api.valueserp.com/batches/{batch_id}/results/1', params)
    api_response = api_result.json()

    while True:
        try:
            request_info = api_response['request_info']
            if request_info['success']:
                break
        except KeyError:
            pass

        time.sleep(3)  # Wait for 1 second before making another request
        api_result = requests.get(f'https://api.valueserp.com/batches/{batch_id}/results/1', params)
        api_response = api_result.json()

    download_links = api_response['result']['download_links']['pages']

    return download_links

def get_search_results(json_url):
    response = requests.get(json_url)

    if response.status_code == 200:
        response_json = response.json()
        return response_json
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def clean_search_results(result_set):
    query_list = []
    links_list = []

    for result_link in result_set:
        response_json = get_search_results(result_link)

        for each in response_json:
            query = each['result']['search_parameters']['q']
            query_list.append(query)

            try:
                links = [link['link'] for link in each['result']['organic_results']]
                links_list.append(links)
            except KeyError:
                links_list.append([])
                print(f"KeyError: 'organic_results' not found in result for query: {query}")

    data = {'query': query_list, 'links': links_list}
    serp_df = pd.DataFrame(data)

    return serp_df

def valueserp_batch_func(df, api_key, batch_name, search_location, gl, hl, google_domain):

  # create batch
  batch_id = create_batch(batch_name, api_key)
  print("Batch ID : " + batch_id)
  time.sleep(1)

  # add search queries
  add_search_queries(batch_id, df, api_key, search_location, gl, hl, google_domain)
  print('Added Search Queries to the ValueSERP Batch')
  time.sleep(1)

  #start the batch
  start_batch(batch_id, api_key)
  print('Started the ValueSERP Batch')
  print('Waiting for SERP Results')

  #get results
  result_set = get_result_set(batch_id, api_key)
  print('SERP Scraping Successful.')

  #clean the results
  cleaned_results = clean_search_results(result_set)
  print('Cleaned Results')

  merged_df = pd.merge(df, cleaned_results, how='left', on='query')
  merged_df = merged_df.rename(columns={'query': 'Keyword', 'impressions': 'Volume', 'links' : 'URLs'})

  return merged_df

In [ ]:
api_key = userdata.get('valueserp_api')

batch_name = 'Inspection Software'
search_location = 'Chicago, Illinois, USA'
gl = 'us'
hl = 'en'
google_domain = 'google.com'

serp_df = valueserp_batch_func(keywords_df, api_key, batch_name, search_location, gl, hl, google_domain)

Batch ID : FBF2E675
{'searches': [{'q': 'inspection software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'safety inspection software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'inspection management software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'quality inspection software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'inspection report software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'quality control inspection software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'car inspection software', 'location': 'Chicago, Illinois, USA', 'gl': 'us', 'hl': 'en', 'google_domain': 'google.com'}, {'q': 'mobile inspection software', 'location': 'Chi

In [ ]:
serp_df

,Keyword,Volume,URLs
0,inspection software,400,[https://www.capterra.com/inspection-software/...
1,inspection software,400,[https://www.capterra.com/inspection-software/...
2,safety inspection software,150,"[https://www.safety-reports.com/, https://safe..."
3,inspection management software,150,[https://www.g2.com/categories/inspection-mana...
4,quality inspection software,150,[https://www.capterra.com/inspection-software/...
...,...,...,...
541,inspection and walkthrough in software testing,0,[https://www.geeksforgeeks.org/difference-betw...
542,sprinkler inspection software download,0,[https://joyfill.io/fire-sprinkler-inspection-...
543,renewable energy inspection software,0,[https://www.fieldeagle.com/renewable-energy-i...
544,railway train inspection software,0,[https://www.railroadsoftware.com/railroad-tra...


In [ ]:
serp_df.to_excel('data.xlsx')

# SERP Based Clustering API

In [ ]:
def get_clusters_from_api(serp_df, common_num=4):
    """
    Sends a POST request to the specified URL with provided SERP DataFrame
    and retrieves clusters DataFrame from the API response.

    Parameters:
        serp_df (pandas.DataFrame): DataFrame containing SERP data.
        common_num (int): Number of common elements in clusters (default is 4).

    Returns:
        pandas.DataFrame: DataFrame containing clusters retrieved from the API.
        None: If an error occurs during the API request.
    """

    # Define the URL and headers
    url = 'https://us-central1-searchblend.cloudfunctions.net/serp-based-clustering'
    headers = {'Content-Type': 'application/json'}

    # Prepare the data for the POST request
    post_data = {
        "serp_df": serp_df.to_dict(),
        "common_num": common_num
    }

    # Convert data to JSON format
    json_data = json.dumps(post_data)

    # Send the POST request
    response = requests.post(url, data=json_data, headers=headers)

    # Check if request was successful
    if response.status_code == 200:
        # Convert response JSON to DataFrame
        clusters_df_cloud = pd.DataFrame(json.loads(response.text))
        return clusters_df_cloud
    else:
        print("Error:", response.status_code)
        return None

In [ ]:
# Example usage:
clusters_df_cloud = get_clusters_from_api(serp_df, common_num=4)
clusters_df_cloud

,Keyword,Volume,Cluster Name,Number of Keywords in Cluster,URLs
0,inspection software,0,inspection software,4,https://www.g2.com/categories/inspection-manag...
1,inspection software,0,inspection software,4,https://www.g2.com/categories/inspection-manag...
2,inspection report software,90,inspection software,4,https://www.softwareadvice.com/home-inspection...
3,inspection software crm,0,inspection software,4,https://www.getapp.com/operations-management-s...
4,safety inspection software,150,safety inspection software,7,"https://goaudits.com/health-safety/,https://ww..."
...,...,...,...,...,...
541,irb inspection software,0,irb inspection software,1,https://apps.apple.com/us/app/irb-inspector-su...
542,pda inspection software,0,pda inspection software,1,https://forum.nachi.org/t/pda-inspection-softw...
543,trakkar inspection software,0,trakkar inspection software,1,"https://inspecttrack.com/,https://appexchange...."
544,sprinkler inspection software download,0,sprinkler inspection software download,1,"https://inspecttrack.com/,https://smartserv.io..."


In [ ]:
clusters_df_cloud.to_excel(batch_name + " clustered.xlsx")

# Visualize Clusters

In [ ]:
import plotly.express as px

# Create the treemap
fig = px.treemap(clusters_df_cloud[clusters_df_cloud['Number of Keywords in Cluster'] > 3],
                 path=['Cluster Name', 'Keyword'])

# Show the treemap
fig.show()